In [1]:
! git clone https://github.com/5amessi/license_plates.git

fatal: destination path 'license_plates' already exists and is not an empty directory.


In [2]:
! git clone https://github.com/SeyedHamidreza/car_plate_dataset.git


fatal: destination path 'car_plate_dataset' already exists and is not an empty directory.


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/car_plate_dataset/IRCP_dataset_1024X768.zip", 'r')
zip_ref.extractall("")
zip_ref.close()

In [4]:
import scipy
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import *
from keras.layers import *


class DataLoader():
    def __init__(self, dataset_name, img_res=(480,240),out_res=(480,240)):
        self.dataset_name = dataset_name
        self.img_res = img_res
        self.out_res = out_res
 
    def load_data(self,png = False):
        if png == True:
          path = glob('/content/%s/*.png' % (self.dataset_name))
        else:
          path = glob('/content/%s/*' % (self.dataset_name))
        
        imgs_hr = []
        imgs_lr = []

        
        for idx , i in enumerate(path):
          if idx >= 1000:
            break
          img = cv2.imread(i)
          w, h = self.img_res
          low_w, low_h = int(w / 4), int(h / 4)

          img_hr = cv2.resize(img, self.out_res)
          
          img_lr = cv2.resize(img, (low_w, low_h))
          img_lr = cv2.resize(img_lr, self.img_res)

          flr=np.fliplr(img_lr)
          fhr=np.fliplr(img_hr)

          imgs_hr.append(img_hr)
          imgs_hr.append(fhr)
          imgs_lr.append(img_lr)
          imgs_lr.append(flr)    

        imgs_hr = np.array(imgs_hr) / 127.5 - 1.
        imgs_lr = np.array(imgs_lr) / 127.5 - 1.

        return imgs_hr, imgs_lr

dl = DataLoader("license_plates")
hr , lr = dl.load_data()
print(np.shape(hr))
print(np.shape(lr))

dl = DataLoader("IRCP_dataset_1024X768")
hr2 , lr2 = dl.load_data()

print(np.shape(hr2))
print(np.shape(lr2))
hr = np.concatenate((hr, hr2))
lr = np.concatenate((lr, lr2))

print(np.shape(hr))
print(np.shape(lr))
t1 = (hr[0] + 1) * 127.5
t1 = np.array(t1, dtype=np.uint8)
cv2.imwrite('hr.jpg',t1)
t1 = (lr[0] + 1) * 127.5
t1 = np.array(t1, dtype=np.uint8)
cv2.imwrite('lr.jpg',t1)


Using TensorFlow backend.


(220, 240, 480, 3)
(220, 240, 480, 3)
(440, 240, 480, 3)
(440, 240, 480, 3)
(660, 240, 480, 3)
(660, 240, 480, 3)


True

In [0]:
def load_image_test(img):
  imgs_lr = []
  w, h = 480,240

  img_lr = cv2.resize(img, (w,h))
  
  imgs_lr.append(img_lr)
  imgs_lr = np.array(imgs_lr) / 127.5 - 1.

  return imgs_lr

In [0]:
def pred(count = 0,idx = 0):
  test = cv2.imread("/content/test.jpg")
  test = load_image_test(test)
  result = model.predict([test])
  result = (result + 1) * 127.5
  result = np.array(result, dtype=np.uint8)
  cv2.imwrite("testout%d.jpg"%(count),result[0])
  
  result = model.predict([[lr[idx]]])
  result = (result + 1) * 127.5
  result = np.array(result, dtype=np.uint8)
  cv2.imwrite("output%d.jpg"%(count),result[0])
  
  result = (lr[idx] + 1) * 127.5
  result = np.array(result, dtype=np.uint8)
  cv2.imwrite("input%d.jpg"%(count),result)
  
  model.save("model%d"%(count))


In [0]:
from keras.engine.saving import load_model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
def Cnn():
    n_residual_blocks = 16

    def residual_block(layer_input, filters):
        d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(layer_input)
        d = BatchNormalization(momentum=0.5)(d)
        d = Activation('relu')(d)
        d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
        d = BatchNormalization(momentum=0.5)(d)
        d = Add()([d, layer_input])
        return d

    img_lr = Input(shape=(240,480,3))

    c1 = Conv2D(64, kernel_size=9, strides=1, padding='same')(img_lr)
    c1 = Activation('relu')(c1)

    r = residual_block(c1, 64)
    for _ in range(n_residual_blocks - 1):
        r = residual_block(r, 64)

    c2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(r)
    c2 = BatchNormalization(momentum=0.5)(c2)
    c2 = Add()([c2, c1])

    gen_hr = Conv2D(3, kernel_size=9, strides=1, padding='same', activation='tanh')(c2)

    return Model(img_lr, gen_hr)


In [8]:
model = Cnn()
optimizer = keras.optimizers.Adam(0.0001)
model.compile(loss='mse',
            optimizer=optimizer)
model.fit(x=lr,y=hr,batch_size=4,epochs=100,verbose=1)
pred(0)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
660/660 [==============================] - 237s 359ms/step - loss: 0.0754
Epoch 2/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0316
Epoch 3/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0286
Epoch 4/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0274
Epoch 5/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0265
Epoch 6/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0262
Epoch 7/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0255
Epoch 8/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0253
Epoch 9/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0249
Epoch 10/100
660/660 [==============================] - 229s 347ms/step - loss: 0.0247
Epoch 11/100
660/66

In [0]:
model.fit(x=lr,y=hr,batch_size=4,epochs=100,verbose=1)
pred(1)


Epoch 1/200
660/660 [==============================] - 229s 347ms/step - loss: 0.0194
Epoch 2/200
660/660 [==============================] - 229s 348ms/step - loss: 0.0218
Epoch 3/200
660/660 [==============================] - 229s 347ms/step - loss: 0.0195
Epoch 4/200
660/660 [==============================] - 229s 347ms/step - loss: 0.0188
Epoch 5/200
660/660 [==============================] - 229s 348ms/step - loss: 0.0184
Epoch 6/200
660/660 [==============================] - 229s 348ms/step - loss: 0.0184
Epoch 7/200
660/660 [==============================] - 230s 348ms/step - loss: 0.0179
Epoch 8/200
660/660 [==============================] - 229s 348ms/step - loss: 0.0179
Epoch 9/200
660/660 [==============================] - 229s 347ms/step - loss: 0.0180
Epoch 10/200
660/660 [==============================] - 228s 345ms/step - loss: 0.0178
Epoch 11/200
660/660 [==============================] - 228s 345ms/step - loss: 0.0178
Epoch 12/200
660/660 [==============================

In [0]:
model.fit(x=lr,y=hr,batch_size=4,epochs=500,verbose=1)
pred(2)

In [0]:
model.fit(x=lr,y=hr,batch_size=4,epochs=1000,verbose=1)
pred(3)


In [0]:
model.fit(x=lr,y=hr,batch_size=4,epochs=1000,verbose=1)
pred(4)


In [0]:
model.fit(x=lr,y=hr,batch_size=4,epochs=1000,verbose=1)
pred(5)


In [0]:
model.fit(x=lr,y=hr,batch_size=4,epochs=1000,verbose=1)
pred(6)